In [1]:
import torch 
from datasets import load_dataset

from transformers import TrainingArguments
from transformers import ImageGPTFeatureExtractor, ImageGPTForCausalImageModeling

/home/aizdebski/micromamba/envs/hybrid-transformer/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def tokenize_function(examples):
    return feature_extractor(examples, return_tensors="pt")

In [3]:
# Load Data
dataset = load_dataset("cifar10")
# tokenized_dataset = [dataset['train']['img'][idx] for idx in range(len(dataset['train']))]


In [6]:
# Prepare batch

tokenized_dataset = [dataset['train']['img'][idx] for idx in range(8)]
inputs = tokenize_function(tokenized_dataset)

In [5]:
SIZE = 'small'
PRETRAINED_MODEL = f'openai/imagegpt-{SIZE}'

feature_extractor = ImageGPTFeatureExtractor.from_pretrained(PRETRAINED_MODEL)
model = ImageGPTForCausalImageModeling.from_pretrained(PRETRAINED_MODEL)

/home/aizdebski/micromamba/envs/hybrid-transformer/lib/python3.11/site-packages/transformers/models/imagegpt/feature_extraction_imagegpt.py:28: FutureWarning: The class ImageGPTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ImageGPTImageProcessor instead.
  warnings.warn(


In [ ]:
training_args = TrainingArguments(output_dir="test_trainer")

dict_keys(['input_ids'])

In [23]:
model.train()
output = model(input_ids=inputs['input_ids'], labels=inputs['input_ids'])

In [46]:
output['logits'].shape

torch.Size([8, 1024, 512])

In [24]:
output.loss

tensor(11.4711, grad_fn=<NllLossBackward0>)

In [14]:
output['logits']

tensor([[[ 1.2487,  1.7292,  2.1103,  ...,  1.6140,  2.9828,  1.9682],
         [ 3.6998,  3.3249,  4.1063,  ...,  4.8907,  3.0700,  4.1013],
         [ 5.9106,  6.1914,  7.0415,  ..., 10.0500,  6.3147,  5.9773],
         ...,
         [ 6.4114,  4.9802,  2.3258,  ...,  9.7507,  5.8174,  5.3543],
         [10.3756, 10.8573,  6.2393,  ..., 15.5685,  9.1309,  9.5843],
         [ 4.9044,  4.7116,  1.8270,  ...,  6.7738,  2.6552,  3.1511]],

        [[ 3.5578,  3.2018,  3.9723,  ...,  4.1569,  4.0747,  3.1275],
         [ 3.1556,  3.5997,  5.5083,  ...,  1.9773,  7.6502,  4.3859],
         [ 6.2591,  7.0568,  7.6928,  ...,  7.1290, 11.8129,  7.0836],
         ...,
         [15.5497, 18.6892,  9.9399,  ..., 19.5463, 19.9757, 19.0096],
         [15.7729, 15.3262,  7.2707,  ..., 15.0155, 17.3975, 15.8299],
         [12.9656, 12.0773,  5.8589,  ..., 13.6987, 12.2500, 11.8931]],

        [[ 7.7896,  8.7571,  7.5025,  ...,  8.8448,  8.7816,  7.3825],
         [31.1267, 34.8285, 22.7777,  ..., 32

In [17]:
shift_logits = output['logits'][..., :-1, :].contiguous()
shift_labels = inputs['input_ids'][..., 1:].contiguous()            

In [16]:
shift_logits.shape

torch.Size([8, 1023, 512])

In [18]:
shift_labels.shape

torch.Size([8, 1023])

In [20]:
loss_fct = CrossEntropyLoss()

In [19]:
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

In [21]:
loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

In [22]:
loss

tensor(11.6928, grad_fn=<NllLossBackward0>)

In [30]:
from tqdm import tqdm 

batch_size = 8

model.train()

# for iteration in tqdm(range(100)):
#     
#     inputs = 

ImageGPTForCausalImageModeling(
  (transformer): ImageGPTModel(
    (wte): Embedding(513, 512)
    (wpe): Embedding(1024, 512)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x ImageGPTBlock(
        (ln_1): ImageGPTLayerNorm()
        (attn): ImageGPTAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): ImageGPTLayerNorm()
        (mlp): ImageGPTMLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): QuickGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): ImageGPTLayerNorm()
  )
  (lm_head): Linear(in_features=512, out_features=512, bias=False)
)

In [26]:
train_dataset = dataset['train']

In [34]:
inputs = train_dataset['img'][index.tolist()]

TypeError: list indices must be integers or slices, not list

In [31]:
import numpy as np

index = np.random.randint(low=0, high=len(train_dataset['img']), size=batch_size)

In [32]:
index

array([37664, 47031, 22036, 26226, 36069, 36596,  1569,  3406])

In [ ]:
x, y = next(iter())
feature_extractor()